# Almond Scala  
Примеры добавления внешних файлов кода  
- ```$ivy``` - пример добавления внешней библиотеки из Maven/Ivy  
- ```$cp``` - пример подключения локального JAR-файла;
- ```$file``` - пример импорта кода из локального файла;
- ```$exec``` - пример выполнения и импорта кода из внешнего скрипта.

https://ammonite.io/

In [1]:
import scala.io.Source
import scala.util.Using
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

import scala.io.Source
import scala.util.Using
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

In [2]:
import scala.sys.process._

import scala.sys.process._

## **$ivy** - пример добавления внешней библиотеки из Maven/Ivy  

Например:  
- требуется прочитать yml - файл в массив структур case class  
Для этого можно использовать библиотеку **snakeyaml** из репозитория Maven https://mvnrepository.com/artifact/org.yaml/snakeyaml  
(Загрузить эту библиотеку из репозитория можно с помощью magic **$ivy**)

In [3]:
// Загружает библиотеку из Maven - репозитория
import $ivy.`org.yaml:snakeyaml:2.3`

import $ivy.$

In [4]:
// Далее простой import
import org.yaml.snakeyaml.Yaml
import java.util.{Map => JMap, List => JList}
import scala.jdk.CollectionConverters._

import org.yaml.snakeyaml.Yaml
import java.util.{Map => JMap, List => JList}
import scala.jdk.CollectionConverters._

### yaml - строка и case class

In [5]:
val ymlString =
  """
entities:
  - entity:
      id: 1
      name: custom_database.example_table_01
      path: /user/hadoop/custom_database.example_table_01
      storage: HIVE
  - entity:
      id: 2
      name: custom_database.example_table_02
      path: /user/hadoop/custom_database.example_table_01
      storage: HIVE
    """.stripMargin
// case class для чтения структуры
case class Entity(
  id: String,
  name: String,
  path: Option[String] = None,
  storage: Option[String] = None 
)

ymlString: String = """
entities:
  - entity:
      id: 1
      name: custom_database.example_table_01
      path: /user/hadoop/custom_database.example_table_01
      storage: HIVE
  - entity:
      id: 2
      name: custom_database.example_table_02
      path: /user/hadoop/custom_database.example_table_01
      storage: HIVE
    """
defined class Entity

### Разбор yaml

In [6]:
// Разбор YAML
val yml = new Yaml()
val parsed_yml = yml.load(ymlString).asInstanceOf[JMap[String, Object]]
// Чтение entities как Scala-Seq
val entitiesDraft = parsed_yml.get("entities").asInstanceOf[JList[JMap[String, Object]]]
val entitiesSeq = entitiesDraft.asScala
// Переобразование в список 
val result = entitiesSeq.map { entityWrapper =>
  val entity = entityWrapper.get("entity").asInstanceOf[JMap[String, Object]].asScala
  entity.toMap
}.toList

yml: Yaml = Yaml:157016332
parsed_yml: java.util.Map[String, Object] = {entities=[{entity={id=1, name=custom_database.example_table_01, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}}, {entity={id=2, name=custom_database.example_table_02, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}}]}
entitiesDraft: java.util.List[java.util.Map[String, Object]] = [{entity={id=1, name=custom_database.example_table_01, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}}, {entity={id=2, name=custom_database.example_table_02, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}}]
entitiesSeq: collection.mutable.Buffer[java.util.Map[String, Object]] = Buffer(
  {entity={id=1, name=custom_database.example_table_01, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}},
  {entity={id=2, name=custom_database.example_table_02, path=/user/hadoop/custom_database.example_table_01, storage=HIVE}}
)
result: List[Map[String, Object]] = 

In [7]:
// Массив case class Entity
val entitiesCaseClass = result.map { m =>
  Entity(
    id = m.get("id").map(_.toString).getOrElse(""),
    name = m.get("name").map(_.toString).getOrElse(""),
    storage = m.get("storage").map(_.toString),
    path = m.get("path").map(_.toString)
  )
}

entitiesCaseClass.foreach(println)

Entity(1,custom_database.example_table_01,Some(/user/hadoop/custom_database.example_table_01),Some(HIVE))
Entity(2,custom_database.example_table_02,Some(/user/hadoop/custom_database.example_table_01),Some(HIVE))


entitiesCaseClass: List[Entity] = List(
  Entity(
    id = "1",
    name = "custom_database.example_table_01",
    path = Some(value = "/user/hadoop/custom_database.example_table_01"),
    storage = Some(value = "HIVE")
  ),
  Entity(
    id = "2",
    name = "custom_database.example_table_02",
    path = Some(value = "/user/hadoop/custom_database.example_table_01"),
    storage = Some(value = "HIVE")
  )
)

### Вывод на экран yaml

In [8]:
entitiesCaseClass.length

res8: Int = 2

In [9]:
println(entitiesCaseClass(0).name)
println(entitiesCaseClass(0).path.getOrElse("") )
println(entitiesCaseClass(0).storage.getOrElse(""))

custom_database.example_table_01
/user/hadoop/custom_database.example_table_01
HIVE


## **$cp** - пример подключения локального JAR-файла  

Например:  
- требуется прочитать xml (и вывести на экран)
Для этого можно использовать библиотеку **jdom2** из репозитория Maven https://mvnrepository.com/artifact/org.jdom/jdom2  
(Необходимо скачать эту библиотеку (через **wget** или самостоятельно) и положить в текущую папку notebook.)  
Подключение библиотеки можно осуществить через magic **$cp** (Classpath) как jar - файл

In [10]:
import scala.sys.process._

import scala.sys.process._

In [11]:
// Скачивание jdom2-2.0.6.1.jar
"rm -f jdom2-2.0.6.1.jar".!!
"wget https://repo1.maven.org/maven2/org/jdom/jdom2/2.0.6.1/jdom2-2.0.6.1.jar".!!

--2025-12-27 09:22:21--  https://repo1.maven.org/maven2/org/jdom/jdom2/2.0.6.1/jdom2-2.0.6.1.jar
Распознаётся repo1.maven.org (repo1.maven.org)… 104.18.19.12, 104.18.18.12, 2606:4700::6812:120c, ...
Подключение к repo1.maven.org (repo1.maven.org)|104.18.19.12|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 327806 (320K) [application/java-archive]
Сохранение в: ‘jdom2-2.0.6.1.jar’

     0K .......... .......... .......... .......... .......... 15% 11,7M 0s
    50K .......... .......... .......... .......... .......... 31% 6,96M 0s
   100K .......... .......... .......... .......... .......... 46% 13,0M 0s
   150K .......... .......... .......... .......... .......... 62% 5,46M 0s
   200K .......... .......... .......... .......... .......... 78% 13,8M 0s
   250K .......... .......... .......... .......... .......... 93% 43,4M 0s
   300K .......... ..........                                 100% 18,3M=0,03s

2025-12-27 09:22:21 (10,5 MB/s) - ‘jdom2-2

res11_0: String = ""
res11_1: String = ""

In [12]:
"test -f jdom2-2.0.6.1.jar".!

res12: Int = 0

In [13]:
// Подключение jar
import $cp.`jdom2-2.0.6.1.jar`
// Import объектов
import org.jdom2.input.SAXBuilder
import java.io.StringReader

import $cp.$
import org.jdom2.input.SAXBuilder
import java.io.StringReader

### XML - строка

In [14]:
val xmlString = """
<entities>
  <entity>
    <id>1</id>
    <name>custom_database.example_table_01</name>
    <path>/user/hadoop/custom_database.example_table_01</path>    
    <path>/user/hadoop/custom_database.example_table_02</path>        
    <storage>HIVE</storage>    
  </entity>
  <entity>
    <id>2</id>
    <name>custom_database.example_table_02</name>
    <path>/user/hadoop/custom_database.example_table_02</path>    
    <storage>HIVE</storage>        
  </entity>
</entities>
"""

xmlString: String = """
<entities>
  <entity>
    <id>1</id>
    <name>custom_database.example_table_01</name>
    <path>/user/hadoop/custom_database.example_table_01</path>    
    <path>/user/hadoop/custom_database.example_table_02</path>        
    <storage>HIVE</storage>    
  </entity>
  <entity>
    <id>2</id>
    <name>custom_database.example_table_02</name>
    <path>/user/hadoop/custom_database.example_table_02</path>    
    <storage>HIVE</storage>        
  </entity>
</entities>
"""

### Разбор XML

In [15]:
val builder = new SAXBuilder()
val document = builder.build(new StringReader(xmlString))
val root = document.getRootElement

// Получаем список элементов <entity>
val entities = root.getChildren("entity").toArray().map(_.asInstanceOf[org.jdom2.Element])

builder: SAXBuilder = org.jdom2.input.SAXBuilder@3ad779dd
document: org.jdom2.Document = [Document:  No DOCTYPE declaration, Root is [Element: <entities/>]]
root: org.jdom2.Element = [Element: <entities/>]
entities: Array[org.jdom2.Element] = Array(
  [Element: <entity/>],
  [Element: <entity/>]
)

### Вывод на экран XML

In [16]:
// Выводим информацию о каждой книге
for (entity <- entities) {
  val id = entity.getChildText("id")
  val name = entity.getChildText("name")
  val path = entity.getChildText("path")  
  val storage = entity.getChildText("storage")   
  println(s"id: $id, name: $name, path: $path, storage: $storage" )
}

id: 1, name: custom_database.example_table_01, path: /user/hadoop/custom_database.example_table_01, storage: HIVE
id: 2, name: custom_database.example_table_02, path: /user/hadoop/custom_database.example_table_02, storage: HIVE


In [17]:
entities.length

res17: Int = 2

## **$file** - пример импорта кода из локального файла

Например:  
- требуется импортировать собственный код для отображения markdown - файла, а затем его использовать
 
Для этого можно использовать magic **\$file**, который импортирует скрипт как модуль:  
- Код выполняется один раз при импорте
- Все определения (функции, переменные, классы) становятся доступны через объект
- Можно обращаться к содержимому через точечную нотацию (или дополнительно сделать ```import <объект>._```)
- 
```Scala
import $file.MyScript

MyScript.elite
// res1: Int = 31337

// Если сценарий находится в подпапке, используйте:
import $file.myfolder.MyScript

// Если сценарий находится во внешней папке:
import $file.^.MyScript

// Если требуется импортировать содержимое скрипта за один раз:
import $file.MyScript, MyScript._
import $file.$

elite
//res1: Int = 31337

// Если требуется импортировать несколько скриптов одновременно
import $file.{MyScript, MyOtherScript}
```

In [18]:
import scala.util.Using
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

import scala.util.Using
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

In [19]:
// Чтение из поддиректории __scalalmond_example_add_external_code скрипта markdown_read_example.sc
import $file.__scalalmond_example_add_external_code.{markdown_read_example => objMD}
//import objMD._

import $file.$

In [20]:
objMD.COLOR

res20: Map[String, String] = HashMap(
  "success" -> "\u001b",
  "warn" -> "\u001b",
  "bold" -> "\u001b",
  "info" -> "\u001b",
  "error" -> "\u001b",
  "header" -> "\u001b",
  "white" -> "\u001b",
  "RESET" -> "\u001b"
)

In [21]:
scala.io.Source

res21: Source.type = scala.io.Source$@25213582

### Чтение файла example.md

In [22]:
// Чтение с ошибкой "файл не найден"
objMD.RichMarkdownReader.displayEnhanced("example.md")


╔══════════════════════════════════════════════════════════════╗
║                 📖 MARKDOWN RICH TEXT                        ║
╚══════════════════════════════════════════════════════════════╝

⏰ Время: 2025-12-27 09:22:26
📁 Файл: example.md

❌ ОШИБКА ЧТЕНИЯ ФАЙЛА

┌──────────────────────────────────────────────────────────────┐
│ example.md (Нет такого файла или каталога)                   │
└──────────────────────────────────────────────────────────────┘


In [23]:
// Вывод файлв на экран (с подсчетом статистик)
objMD.RichMarkdownReader.displayEnhanced("__scalalmond_example_add_external_code/example.md")


╔══════════════════════════════════════════════════════════════╗
║                 📖 MARKDOWN RICH TEXT                        ║
╚══════════════════════════════════════════════════════════════╝

⏰ Время: 2025-12-27 09:22:26
📁 Файл: __scalalmond_example_add_external_code/example.md

✅ ФАЙЛ УСПЕШНО ПРОЧИТАН

📊 СТАТИСТИКА:
┌────────────────────┬──────────┐
│ Кол-во символов    │     3449 │
│ Кол-во строк       │       45 │
│ Кол-во заголовков  │        9 │
│ Кол-во блоков      │        0 │
│ Кол-во ссылок      │       13 │
│ Кол-во изображений │        0 │
└────────────────────┴──────────┘

👀 ПРЕВЬЮ СОДЕРЖИМОГО:
════════════════════════════════════════════════════════════
🔥 PyTorch Android Examples

   A list of Android demo apps built on the powerful [PyTorch Mobile](https://pytorch.org/mobile) platform.

💡 HelloWorld

   [HelloWorld](https://github.com/pytorch/android-demo-app/tree/master/HelloWorldApp) is a simple image classification application that demonstrates how to use the PyTor

## **$exec** - пример импорта кода из локального файла

Например:  
- требуется импортировать собственный код функциями для работы с **html**, а затем его использовать

Magic **\$exec** - просто выполняет скрипт:
- Код выполняется в текущем контексте
- Все определения становятся доступны напрямую (без префикса)
- Похоже на copy-paste кода

import **\$exec** похож на import **\$file**, за исключением того, что он сбрасывает определения и импорт из сценария в текущую сессию REPL. То есть можно сделать некоторый набор функций, которые при импорте будут доступный в сессии Jupyter Notebook ASIS. Все команды и определения из этого скрипта будут выполнены так, как если бы их ввели непосредственно в текущую сессию. Это полезно для интеграции и выполнения кода из внешних файлов. Код исполняется в текущем контексте, и любые изменения, сделанные в скрипте, влияют на текущую сессию Ammonite.

In [24]:
import $exec.__scalalmond_example_add_external_code.{html_code_example => _}

Compiling /home/chul/WORKSHOP/SCALA_ALMOND/__scalalmond_example_add_external_code/html_code_example.sc


import $exec.$

### Прстой вывод текста в html - формате

In [25]:
html_print("Результаты анализа", "Данные обработаны успешно")

Результаты анализа 
 Данные обработаны успешно

In [26]:
val headers = Seq("Название", "Цена", "Тип торговой точки")
val data = Seq(
  Seq("Яблоки", 10, "Магазин" ),
  Seq("Груши", 15, "Рынок"),
  Seq("Апельсины", 8, "Магазин" )
)

headers: Seq[String] = List("Название", "Цена", "Тип торговой точки")
data: Seq[Seq[Any]] = List(
  List("Яблоки", 10, "Магазин"),
  List("Груши", 15, "Рынок"),
  List("Апельсины", 8, "Магазин")
)

### Таблица в html - формате

In [27]:
html_table(headers, data, "15%")

Название,Цена,Тип торговой точки
Яблоки,10,Магазин
Груши,15,Рынок
Апельсины,8,Магазин


In [28]:
val headers = Seq("Название")
val data = Seq(
  Seq("Яблоки"),
  Seq("Груши"),
  Seq("Апельсины" )
)

headers: Seq[String] = List("Название")
data: Seq[Seq[String]] = List(List("Яблоки"), List("Груши"), List("Апельсины"))

In [29]:
html_table(headers, data, "15%")

Название
Яблоки
Груши
Апельсины


### html - отчет

In [30]:
html_report(reportH = "Какая то статистика", Map("Январь" -> 100.5, "Февраль" -> 120.3, "Март" -> 95.8) )

Какая то статистика 
 
 Январь: 
 100,50 
 
 
 Февраль: 
 120,30 
 
 
 Март: 
 95,80